<a href="https://colab.research.google.com/github/hwankang/Deep-Chemometrics/blob/master/27(fri)_Jan_8_00_pm_sgf_normal_Raman_R2_992_size_0_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#The program of deep learning with Raman data

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import scipy
from scipy.signal import savgol_filter
from sklearn.model_selection import train_test_split, cross_validate

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
filename_a='/content/drive/MyDrive/machine_learning/123_Raman_DATA_csv.csv'
X2_df = pd.read_csv(filename_a)
X2_df

In [ ]:
X=X2_df.values[:,3:].astype(float)
#X_train=X[110:,:]
#X_test=X[0:110,:]
y=X2_df.values[:,2:5].astype(float)
#y_train=Y[110:,:]
#y_test=Y[0:110,:]
#X_train
X_train, X_test, y_train, y_test = train_test_split(
     X, y, test_size=0.1, random_state=50)

In [ ]:
#from scipy.signal import savgol_filter
#X_train_f=X_train_n.astype(float)
#X_test_f=X_test_n.astype(float)
X_train_s = savgol_filter(X_train, 17, polyorder=2, deriv=1)
X_test_s = savgol_filter(X_test, 17, polyorder=2, deriv=1)

In [ ]:

def snv(input_data):
  
    # Define a new array and populate it with the corrected data  
    output_data = np.zeros_like(input_data)
    for i in range(input_data.shape[0]):
 
        # Apply correction
        output_data[i,:] = (input_data[i,:] - np.mean(input_data[i,:])) / np.std(input_data[i,:])
 
    return output_data

In [ ]:
X_train_r=snv(X_train_s)
X_test_r=snv(X_test_s)

In [ ]:
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv1D, Reshape#, MaxPooling1D
from keras.callbacks import ReduceLROnPlateau
from keras.layers.noise import GaussianNoise

In [ ]:
DENSE = 128
#DROPOUT = 0.025
DROPOUT = 0
C1_K  = 8 #Number of kernels/feature extractors for first layer
C1_S  = 32 #Width of the convolutional mini networks
C2_K  = 16
C2_S  = 32

#activation='elu'
activation='relu'
#activation='sigmoid'

input_dim = X_train.shape[1]

#The model
def make_model():
    model = Sequential()
    #Adding a bit of GaussianNoise also works as regularization
    #model.add(GaussianNoise(0.05, input_shape=(input_dim,)))
    model.add(GaussianNoise(0.005, input_shape=(input_dim,)))
    ##model.add(GaussianNoise(0.004, input_shape=(input_dim,)))
    #First two is number of filter + kernel size
    model.add(Reshape((input_dim, 1)))
    #model.add(Conv1D(C1_K, (C1_S), activation=activation, border_mode='same'))
    #model.add(Conv1D(C2_K, (C2_S), border_mode='same', activation=activation))
    model.add(Conv1D(C1_K, (C1_S), activation=activation, padding='same'))
    model.add(Conv1D(C2_K, (C2_S), padding='same', activation=activation))
    model.add(Flatten())
    model.add(Dropout(DROPOUT))
    model.add(Dense(DENSE, activation=activation))
    model.add(Dense(3, activation='linear'))

    #model.compile(loss='mse', optimizer=keras.optimizers.Adadelta(lr=0.01))#, metrics=['mean_absolute_error'])
    model.compile(loss='mse', optimizer=keras.optimizers.Adam(lr=0.001))#, metrics=['mean_absolute_error'])

    return model

In [ ]:
model = make_model()
print(model.summary())

In [ ]:
rdlr = ReduceLROnPlateau(patience=25, factor=0.5, min_lr=1e-6, monitor='val_loss', verbose=1)

h = model.fit(X_train_r, y_train, epochs=200, batch_size=18*2, validation_data=(X_test_r, y_test), callbacks=[rdlr])

In [ ]:
plt.plot(h.history['loss'], label='loss')
plt.plot(h.history['val_loss'], label='val_loss')

plt.yscale('log')
plt.ylabel('Loss')
plt.xlabel('Epochs')
plt.legend()
ax2 = plt.gca().twinx()
ax2.plot(h.history['lr'], color='r')
ax2.set_ylabel('lr',color='r')

_ = plt.legend()

In [ ]:
plt.scatter(y_train, model.predict(X_train_r))
plt.scatter(y_test, model.predict(X_test_r))
plt.plot([-5,3],[-5,3]) # Y = PredY line

In [ ]:
pred_Y_df=model.predict(X_test_r)
Y_df_true=y_test
from sklearn.metrics import mean_squared_error, r2_score
mse= mean_squared_error(Y_df_true, pred_Y_df)
rmse=np.sqrt(mse)
r2=r2_score(Y_df_true, pred_Y_df)
print('MSE : {: 0.3f}, || RMSE : {: 0.3f}, || R2 : {: 0.3f}'.format(mse,rmse,r2))

#end of the program